In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

df = pd.read_csv("logement_tana_avec_idquartier.csv")

print("Aperçu du dataset :")
print(df.head())

print("\nValeurs manquantes par colonne :")
print(df.isnull().sum())
num_cols = ['superficie', 'surface_terrain', 'nombre_chambres', 'nombre_salles_eau'] 
cat_cols = ['quartier', 'douche_wc', 'etat_general', 'type_d_acces', 'type_logement', 'meuble'] 

target_col = 'loyer_mensuel' 

num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer(transformers=[
    ('num', Pipeline([
        ('imputer', num_imputer),
        ('scaler', StandardScaler())
    ]), num_cols),

    ('cat', Pipeline([
        ('imputer', cat_imputer),
        ('encoder', encoder)
    ]), cat_cols)
])

X = df.drop(columns=[target_col])
y = df[target_col]

X_preprocessed = preprocessor.fit_transform(X)

print("\n✅ Prétraitement terminé. Dimensions de X prétraité :", X_preprocessed.shape)


Aperçu du dataset :
   a_etage  cheminee classification_f  cuve_eau  douche_wc etat_general  \
0    False      True               F6     False  interieur          bon   
1    False     False               F5     False  interieur          bon   
2    False     False               F4     False  interieur          bon   
3    False     False               F6     False  interieur          bon   
4    False     False               F6     False  interieur          bon   

   garage_ferme  gardien_securite  groupe_electrogene  loyer_mensuel  ...  \
0         False             False               False      4000000.0  ...   
1         False              True               False      9000000.0  ...   
2         False             False               False      1700000.0  ...   
3         False             False               False      8500000.0  ...   
4         False             False               False      8500000.0  ...   

   nombre_salles_eau  piscine                   quartier  superfic

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import VarianceThreshold
import warnings

warnings.filterwarnings(action='ignore', category=RuntimeWarning)

df = pd.read_csv("logement_cleaned.csv")

# Convertir les colonnes booléennes en int
bool_cols = [col for col in df.columns if df[col].dtype == "bool"]
for col in bool_cols:
    df[col] = df[col].astype(int)

# Définir les colonnes numériques et catégorielles
num_cols = ["superficie", "surface_terrain", "nombre_chambres", "nombre_salles_eau"] + bool_cols
cat_cols = ["quartier", "douche_wc", "etat_general", "type_d_acces", "type_logement"]

X = df.drop(columns=["loyer_mensuel"])
y = df["loyer_mensuel"]

# Filtrage par variance
X_num = X[num_cols].copy()
imputer_num = SimpleImputer(strategy="median")
X_num_imputed = pd.DataFrame(imputer_num.fit_transform(X_num), columns=num_cols)

selector_var = VarianceThreshold(threshold=1e-5)
selector_var.fit(X_num_imputed)
cols_var = X_num_imputed.columns[selector_var.get_support()].tolist()

num_cols_filtered = cols_var

# Séparation en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipelines pour les colonnes numériques et catégorielles
num_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", num_transformer, num_cols_filtered),
    ("cat", cat_transformer, cat_cols)
])

# Pipeline complet
pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("select", SelectKBest(score_func=f_regression)),
    ("regressor", Ridge())
])

# Grille des hyperparamètres
param_grid = {
    "select__k": [10, 20, 30, 40, 50],
    "regressor__alpha": [0.1, 1.0, 10.0, 100.0]
}

# GridSearch sans verbose
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    scoring="r2",
    cv=5,
    n_jobs=-1
)

# Entraînement
grid_search.fit(X_train, y_train)

# Prédiction et évaluation
y_pred = grid_search.best_estimator_.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f" R² test final : {r2:.4f}")
print(f" RMSE test final : {rmse:.2f}")


 R² test final : 0.7284
 RMSE test final : 1724018.21


In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

df = pd.read_csv("logement_cleaned.csv")

bool_cols = [col for col in df.columns if df[col].dtype == "bool"]
for col in bool_cols:
    df[col] = df[col].astype(int)

num_cols = ["superficie", "surface_terrain", "nombre_chambres", "nombre_salles_eau"] + bool_cols
cat_cols = ["quartier", "douche_wc", "etat_general", "type_d_acces", "type_logement"]

X = df.drop(columns=["loyer_mensuel"])
y = df["loyer_mensuel"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

num_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
cat_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)) 
])
preprocessor = ColumnTransformer([
    ("num", num_transformer, num_cols),
    ("cat", cat_transformer, cat_cols)
])

X_train_preprocessed = preprocessor.fit_transform(X_train)

X_train_sm = sm.add_constant(X_train_preprocessed)

cols = list(range(X_train_sm.shape[1]))

def backward_elimination(X, y, cols, sl=0.05):
    """
    Backward elimination basée sur p-values.
    X : array numpy (avec constante)
    y : target
    cols : liste indices des colonnes actuellement retenues
    sl : seuil de significativité
    """
    while True:
        X_opt = X[:, cols]
        model = sm.OLS(y, X_opt).fit()
        pvalues = model.pvalues
        max_pval = max(pvalues)
        if max_pval > sl:
            max_pval_index = pvalues.argmax()
            print(f"Suppression variable idx={cols[max_pval_index]} avec p-value={max_pval}")
            cols.pop(max_pval_index)
        else:
            break
    return cols, model

cols_selected, model_final = backward_elimination(X_train_sm, y_train.values, cols)

print("\nVariables retenues après backward elimination :", cols_selected)
print(model_final.summary())

X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

ridge = Ridge(alpha=1.0)
n_features_to_select = 20  

selector = RFE(estimator=ridge, n_features_to_select=n_features_to_select, step=1)
selector = selector.fit(X_train_transformed, y_train)

print(f"\nNombre de features sélectionnées par RFE : {sum(selector.support_)}")

X_train_rfe = selector.transform(X_train_transformed)
X_test_rfe = selector.transform(X_test_transformed)

ridge.fit(X_train_rfe, y_train)
y_pred_rfe = ridge.predict(X_test_rfe)

r2 = r2_score(y_test, y_pred_rfe)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rfe))

print(f"RFE + Ridge - R² sur test : {r2:.4f}")
print(f"RFE + Ridge - RMSE sur test : {rmse:.2f}")

print("Indices des features retenues par RFE :", np.where(selector.support_)[0])


Suppression variable idx=4 avec p-value=0.9911521483560483
Suppression variable idx=16 avec p-value=0.962720231695096
Suppression variable idx=6 avec p-value=0.9419752341254297
Suppression variable idx=30 avec p-value=0.941842603052384
Suppression variable idx=12 avec p-value=0.8425156178884275
Suppression variable idx=32 avec p-value=0.8152850361311793
Suppression variable idx=35 avec p-value=0.8763565531619564
Suppression variable idx=36 avec p-value=0.8987591723250102
Suppression variable idx=8 avec p-value=0.7527384438206777
Suppression variable idx=20 avec p-value=0.6749435111378066
Suppression variable idx=33 avec p-value=0.6534993147593869
Suppression variable idx=27 avec p-value=0.7741444263628234
Suppression variable idx=19 avec p-value=0.5844077682144027
Suppression variable idx=14 avec p-value=0.4996760834349022
Suppression variable idx=28 avec p-value=0.42195835429374084
Suppression variable idx=31 avec p-value=0.3617378816879484
Suppression variable idx=24 avec p-value=0.2


# TP : Prédire le prix de location de logements à Antananarivo avec une régression linéaire multiple

## 🎯 Objectifs pédagogiques

- Appliquer un pipeline de prétraitement complet sur un jeu de données semi-structuré.
- Gérer les variables qualitatives, les valeurs manquantes, la multicolinéarité et la scalabilité.
- Construire, tester et évaluer un modèle de régression linéaire multiple.
- Déployer le modèle dans une application Python Streamlit avec interface utilisateur.

## 🗂️ Jeu de données

Le jeu de données doit être collecté ou scrappé dans les pages comme Facebook. Il doit comporter les colonnes suivantes :

- `quartier` (catégorielle)
- `superficie` (numérique)
- `nombre_chambres` (numérique)
- `douche_wc`(interieur ou exterieur)
- `type_d_acces` (sans, moto, voiture, voiture_avec_par_parking)
- `meublé` (booléen:  oui ou non)
- `état_général` (catégorielle : bon, moyen, mauvais)
- `loyer_mensuel` (target)

## 🧪 Étapes du TP

### 📌 Partie 1 : Préparation des données
- Lecture du dataset brut
- Gestion des valeurs manquantes
- Encodage des variables catégorielles
- Création de variables dérivées
- Détection et suppression des variables fortement corrélées
- Standardisation et normalisation

### 📌 Partie 2 : Modélisation

- Séparation train/test
- Implémentation de la régression linéaire multiple
- Évaluation : R², RMSE
- Vérification des hypothèses d'élligibilité de la régression linéaire multiple (surtout sur les erreurs)

### 📌 Partie 3 : Optimisation du modèle

- Sélection de variables : backward elimination, RFE (à documenter)

### 📌 Partie 4 : Déploiement d’une application Streamlit

- Interface de saisie utilisateur
- Affichage du loyer prédit
- Visualisation des poids des variables
- Affichage sur la carte interactive

## 🧭 Carte interactive (option avancée)

Utiliser `streamlit-folium` pour permettre à l’utilisateur de cliquer sur une carte et de récupérer les coordonnées GPS. À partir de ces coordonnées, déterminer automatiquement le quartier en utilisant un fichier GeoJSON ou un système de polygones avec `geopandas`.

## 🔧 Technologies à utiliser

- `pandas`, `numpy`, `scikit-learn`, `matplotlib`, `seaborn`, `joblib`
- `streamlit`, `folium`, `streamlit-folium`
- Optionnel : `geopandas`, `shapely`

## 💡 Bonus

- Carte interactive avec folium
- Simuler des données additionnelles (pollution, sécurité)
- Tri automatique des caractéristiques influentes


